In [ ]:
import json
import numpy as np
import pandas as pd
from IPython.display import display

In [ ]:
# Configurazione opzioni Pandas e Matplotlib

pd.set_option('display.max_columns', None)

In [ ]:
# Leggo il file di configurazione
with open('config/config.json') as f:
    config_data = json.load(f)

# Ottengo i percorsi dei file
file_path_vaccinazioni_covid = config_data['path_covid_19_france_vaccinations_age_sexe_dep']
file_path_andamento_dei_prezzi_al_consumo = config_data['path_economicref_france_indice_prix_consommation']
file_path_risultati_delle_elezioni_legislative_1_turno = config_data['path_elections_france_legislatives_2022_1er_tour_par_bureau_de_vote']
file_path_risultati_delle_elezioni_legislative_2_turno = config_data['path_elections_france_legislatives_2022_2nd_tour_par_bureau_de_vote']
file_path_risultati_delle_elezioni_presidenziali_1_turno = config_data['path_elections_france_presidentielles_2022_1er_tour_par_bureau_de_vote']
file_path_risultati_delle_elezioni_presidenziali_2_turno = config_data['path_elections_france_presidentielles_2022_2nd_tour_par_bureau_de_vote']
file_path_dati_sulla_disoccupazione = config_data['path_labouref_france_departement_quarter_jobseeker']
file_path_andamento_dei_prezzi_dei_carburanti = config_data['path_prix_des_carburants_j_1_001']
file_path_reddito_per_dipartimenti = config_data['path_departements_plus_riches_de_france']
file_path_inflazione = config_data['path_inflazione']

file_path_work_vaccinazioni_covid = config_data['path_work_vaccinazioni_covid']
file_path_work_andamento_prezzi_consumo = config_data['path_work_andamento_prezzi_consumo']
file_path_work_risultati_delle_elezioni_legislative_1_turno = config_data['path_work_risultati_elezioni_legislative_1_turno']
file_path_work_risultati_delle_elezioni_legislative_2_turno = config_data['path_work_risultati_elezioni_legislative_2_turno']
file_path_work_risultati_delle_elezioni_presidenziali_1_turno = config_data['path_work_risultati_elezioni_presidenziali_1_turno']
file_path_work_risultati_delle_elezioni_presidenziali_2_turno = config_data['path_work_risultati_elezioni_presidenziali_2_turno']
file_path_work_dati_sulla_disoccupazione = config_data['path_work_dati_sulla_disoccupazione']
file_path_work_andamento_dei_prezzi_dei_carburanti = config_data['path_work_andamento_dei_prezzi_dei_carburanti']
file_path_work_reddito_per_dipartimenti = config_data['path_work_reddito_per_dipartimenti']
file_path_work_inflazione = config_data['path_work_inflazione']

In [ ]:
# Filtro dataset per regioni(solo regioni assegnate al nostro gruppo)

def filtra_dataset_per_regioni(file_path_dataset, sep=';', dtype={}):
    df_dataset = pd.read_csv(file_path_dataset, sep=sep, dtype=dtype)
    mask = (df_dataset['Nom Officiel Région'] == 'Hauts-de-France') | \
           (df_dataset['Nom Officiel Région'] == 'Grand Est') | \
           (df_dataset['Nom Officiel Région'] == 'Île-de-France') | \
           (df_dataset['Nom Officiel Région'] == 'Normandie') | \
           (df_dataset['Nom Officiel Région'] == 'Bretagne') | \
           (df_dataset['Nom Officiel Région'] == 'Bourgogne-Franche-Comté') | \
           (df_dataset['Nom Officiel Région'] == 'Centre-Val de Loire')
    df_dataset = df_dataset[mask]
    df_dataset.reset_index(drop=True, inplace=True)
    return df_dataset

In [ ]:
# Definizione funzioni ausiliarie (Cleaning)

def float_converter(s):
    s = s.replace(",", ".")
    return s

def thousands_converter(s):
    s = s.replace(",", "")
    return s

def substring_datetime(s):
    s = s.split('T')[0]
    return s

In [ ]:
# Caricamento e cleaning dataset vaccinazioni_covid

df_vaccinazioni_covid = filtra_dataset_per_regioni(file_path_vaccinazioni_covid,
                                                   dtype={"Code Officiel Département": str})
df_vaccinazioni_covid.rename(columns={"Date": "data", "Nombre de doses n°1": "numero_prime_dosi", "Nombre cumulé de doses n°1":"numero_cumulativo_prime_dosi",
                                      "Type de variable":"tipo_variabile","Valeur de la variable":"valore_variabile","Libellé de la variable":"etichetta_variabile",
                                      "Nom Officiel Département":"nome_dipartimento", "Code Officiel Région":"codice_regione", "Nom Officiel Région":"nome_regione",
                                      "Code Officiel Département":"codice_dipartimento", "Nombre de vaccinations complètes (doses n°2)":"numero_seconde_dosi",
                                      "Nombre de personnes vaccinées avec une dose de rappel (3e dose)":"numero_terze_dosi",
                                      "Nombre de personnes vaccinées avec une deuxième dose de rappel (4e dose)":"numero_quarte_dosi",
                                      "Nombre de doses de rappel bivalent":"numero_dosi_bivalenti",
                                      "Nombre de personnes ayant reçu une troisième dose de rappel":"numero_persone_terza_dose",
                                      "Nombre cumulé de vaccinations complètes (doses n°2)":"numero_cumulativo_seconde_dosi",
                                      "Nombre cumulé de personnes vaccinées avec une dose de rappel (3e dose)":"numero_cumulativo_terze_dosi",
                                      "Nombre cumulé de personnes vaccinées avec une deuxième dose de rappel (4e dose)":"numero_cumulativo_quarte_dosi",
                                      "Nombre cumulé de personnes vaccinées avec une dose de rappel bivalent":"numero_cumulativo_dosi_bivalenti",
                                      "Nombre cumulé de personnes ayant reçu une troisième dose de rappel":"numero_cumulativo_persone_terza_dose_richiamo",
                                      "Couverture vaccinale partielle (dose n°1)":"copertura_vaccinale_prima_dose",
                                      "Couverture vaccinale complète (dose n°2)":"copertura_vaccinale_seconda_dose",
                                      "Couverture vaccinale rappel/3e dose":"copertura_vaccinale_terza_dose",
                                      "Couverture vaccinale 2e rappel/4e dose":"copertura_vaccinale_quarta_dose",
                                      "Couverture vaccinale du rappel bivalent":"copertura_vaccinale_dose_bivalente",
                                      "Couverture vaccinale de la troisième dose de rappel":"copertura_vaccinale_terza_dose_richiamo"}, inplace=True)
df_vaccinazioni_covid.drop(['numero_persone_terza_dose','numero_cumulativo_persone_terza_dose_richiamo', 'copertura_vaccinale_terza_dose_richiamo'],
                           axis=1, inplace=True)
new_ordered_columns = ["data","nome_regione","codice_regione","nome_dipartimento","codice_dipartimento","tipo_variabile","etichetta_variabile","valore_variabile",
                       "numero_prime_dosi","numero_seconde_dosi","numero_terze_dosi","numero_quarte_dosi","numero_cumulativo_prime_dosi",
                       "numero_cumulativo_seconde_dosi","numero_cumulativo_terze_dosi","numero_cumulativo_quarte_dosi","numero_dosi_bivalenti",
                       "numero_cumulativo_dosi_bivalenti", "copertura_vaccinale_prima_dose", "copertura_vaccinale_seconda_dose", "copertura_vaccinale_terza_dose",
                       "copertura_vaccinale_quarta_dose", "copertura_vaccinale_dose_bivalente"]
df_vaccinazioni_covid = df_vaccinazioni_covid.reindex(columns=new_ordered_columns)
df_vaccinazioni_covid.to_csv(file_path_work_vaccinazioni_covid, index=False)
#df_vaccinazioni_covid.head(10)

In [ ]:
# Caricamento e cleaning dataset andamento_prezzi_consumo

df_andamento_prezzi_consumo = pd.read_csv(file_path_andamento_dei_prezzi_al_consumo, sep=';')
df_andamento_prezzi_consumo.rename(columns={"Période" : "periodo", "Indice" : "suggerimento", "Valeur": "valore","Code" : "codice","Type" : "tipo",
                                            "Sous-type" : "sottotipo", "Base" : "base", "Population" : "popolazione", "Zone concernée" : "zona_interessata",
                                            "Code COICOP" : "codice_COICOP", "Désignation COICOP" : "designazione_COICOP", "Série arrêtée" : "serie_interrotta",
                                            "Référence INSEE" : "riferimento_INSEE"}, inplace=True)
df_andamento_prezzi_consumo['periodo'] = pd.to_datetime(df_andamento_prezzi_consumo['periodo'])
df_andamento_prezzi_consumo = df_andamento_prezzi_consumo[((df_andamento_prezzi_consumo['periodo'].dt.year == 2022) &
                                                          (df_andamento_prezzi_consumo['periodo'].dt.month >= 4)) |
                                                          (df_andamento_prezzi_consumo['periodo'].dt.year == 2023)]
df_andamento_prezzi_consumo.to_csv(file_path_work_andamento_prezzi_consumo, index=False)
#df_andamento_prezzi_consumo.head(10)

In [ ]:
# Caricamento e cleaning dataset risultati_elezioni_legislative_1_turno

df_risultati_elezioni_legislative_1_turno = filtra_dataset_per_regioni(file_path_risultati_delle_elezioni_legislative_1_turno,
                                                                       dtype={"Code du département": str, "Code du b.vote": str,
                                                                              "Code de la commune": str, "Exprimés": int})
df_risultati_elezioni_legislative_1_turno.rename(columns={"Code du département" : "codice_dipartimento", "Libellé du département" : "nome_dipartimento",
                                                          "Code de la circonscription" : "codice_della_circoscrizione",
                                                          "Libellé de la circonscription" : "nome_della_circoscrizione", "Code de la commune" : "codice_comune",
                                                          "Libellé de la commune" : "nome_del_comune", "Code du b.vote" : "codice_seggio_elettorale",
                                                          "Inscrits" : "iscritti", "Abstentions" : "astenuti", "% Abs/Ins" : "% astenuti/iscritti",
                                                          "Votants" : "votanti", "% Vot/Ins" : "% votanti/iscritti", "Blancs" : "bianche",
                                                          "% Blancs/Ins" : "% bianche/iscritti", "% Blancs/Vot" : "% bianche/votanti",
                                                          "Nuls" : "nulle", "% Nuls/Ins" : "% nulle/iscritti", "% Nuls/Vot" : "% nulle/votanti",
                                                          "Exprimés" : "espressi", "% Exp/Ins" : "% espressi/iscritti", "% Exp/Vot" : "% espressi/votanti",
                                                          "N°Panneau" : "pannello_numero", "Sexe" : "genere", "Nom" : "cognome", "Prénom" : "nome",
                                                          "Nuance" : "partito", "Voix" : "voti", "% Voix/Ins" : "% voti/iscritti", "% Voix/Exp" : "% voti/iscritti",
                                                          "Code Officiel EPCI" : "codice_ufficiale_EPCI", "Nom Officiel EPCI" : "nome_ufficiale_dello_EPCI",
                                                          "Code Officiel Région" : "codice_ufficiale_regionale", "Nom Officiel Région" : "nome_ufficiale_regionale",
                                                          "location" : "coordinate", "lib_du_b_vote" : "etichetta_del_seggio_elettorale",
                                                          "scrutin_code" : "codice_scrutinio"}, inplace=True)
df_risultati_elezioni_legislative_1_turno.to_csv(file_path_work_risultati_delle_elezioni_legislative_1_turno, index=False)
#df_risultati_elezioni_legislative_1_turno.head(10)

In [ ]:
# Caricamento e cleaning dataset risultati_elezioni_legislative_2_turno

df_risultati_elezioni_legislative_2_turno = filtra_dataset_per_regioni(file_path_risultati_delle_elezioni_legislative_2_turno,
                                                                       dtype={"Code du département": str, "Code du b.vote": str,
                                                                              "Code de la commune": str, "Exprimés": int})
df_risultati_elezioni_legislative_2_turno.rename(columns={"Code du département" : "codice_dipartimento", "Libellé du département" : "nome_dipartimento",
                                                          "Code de la circonscription" : "codice_della_circoscrizione",
                                                          "Libellé de la circonscription" : "nome_della_circoscrizione", "Code de la commune" : "codice_comune",
                                                          "Libellé de la commune" : "nome_del_comune", "Code du b.vote" : "codice_seggio_elettorale",
                                                          "Inscrits" : "iscritti", "Abstentions" : "astenuti", "% Abs/Ins" : "% astenuti/iscritti",
                                                          "Votants" : "votanti", "% Vot/Ins" : "% votanti/iscritti",
                                                          "Blancs" : "bianche", "% Blancs/Ins" : "% bianche/iscritti", "% Blancs/Vot" : "% bianche/votanti",
                                                          "Nuls" : "nulle", "% Nuls/Ins" : "% nulle/iscritti", "% Nuls/Vot" : "% nulle/votanti",
                                                          "Exprimés" : "espressi", "% Exp/Ins" : "% espressi/iscritti", "% Exp/Vot" : "% espressi/votanti",
                                                          "N°Panneau" : "pannello_numero", "Sexe" : "genere", "Nom" : "cognome", "Prénom" : "nome",
                                                          "Nuance" : "partito", "Voix" : "voti", "% Voix/Ins" : "% voti/iscritti", "% Voix/Exp" : "% voti/iscritti",
                                                          "Code Officiel EPCI" : "codice_ufficiale_EPCI", "Nom Officiel EPCI" : "nome_ufficiale_dello_EPCI",
                                                          "Code Officiel Région" : "codice_ufficiale_regionale", "Nom Officiel Région" : "nome_ufficiale_regionale",
                                                          "location" : "coordinate", "lib_du_b_vote" : "etichetta_del_seggio_elettorale",
                                                          "scrutin_code" : "codice_scrutinio"}, inplace=True)
df_risultati_elezioni_legislative_2_turno.to_csv(file_path_work_risultati_delle_elezioni_legislative_2_turno, index=False)
#df_risultati_elezioni_legislative_2_turno.head(10)

In [ ]:
# Caricamento e cleaning dataset risultati_elezioni_presidenziali_1_turno

df_risultati_elezioni_presidenziali_1_turno = filtra_dataset_per_regioni(file_path_risultati_delle_elezioni_presidenziali_1_turno,
                                                                         dtype={"Code du département": str, "Code de la circonscription": str,
                                                                                "Code de la commune": str, "Code du b.vote": str, "Code Officiel EPCI": str,
                                                                                "Exprimés": int})
df_risultati_elezioni_presidenziali_1_turno['% Voix/Exp'] = df_risultati_elezioni_presidenziali_1_turno['% Voix/Exp'].apply(float_converter)
df_risultati_elezioni_presidenziali_1_turno['% Voix/Exp'] = df_risultati_elezioni_presidenziali_1_turno['% Voix/Exp'].astype(float)
df_risultati_elezioni_presidenziali_1_turno.rename(columns={"Code du département" : "codice_dipartimento", "Libellé du département" : "nome_dipartimento",
                                                            "Code de la circonscription" : "codice_circoscrizione",
                                                            "Libellé de la circonscription" : "nome_circoscrizione", "Code de la commune" : "codice_comune",
                                                            "Libellé de la commune" : "nome_comune", "Code du b.vote" : "codice_seggio_elettorale",
                                                            "Inscrits" : "iscritti", "Abstentions" : "astenuti", "% Abs/Ins" : "% astenuti/iscritti",
                                                            "Votants" : "votanti", "% Vot/Ins" : "% votanti/iscritti", "Blancs" : "bianche",
                                                            "% Blancs/Ins" : "% bianche/iscritti", "% Blancs/Vot" : "% bianche/votanti",
                                                            "Nuls" : "nulle", "% Nuls/Ins" : "% nulle/iscritti", "% Nuls/Vot" : "% nulle/votanti",
                                                            "Exprimés" : "espressi", "% Exp/Ins" : "% espressi/iscritti", "% Exp/Vot" : "% espressi/votanti",
                                                            "N°Panneau" : "pannello_numero", "Sexe" : "genere", "Nom" : "cognome", "Prénom" : "nome",
                                                            "Nuance" : "partito", "Voix" : "voti", "% Voix/Ins" : "% voti/iscritti", "% Voix/Exp" : "% voti/espressi",
                                                            "Code Officiel EPCI" : "codice_EPCI", "Nom Officiel EPCI" : "nome_EPCI",
                                                            "Code Officiel Région" : "codice_regione", "Nom Officiel Région" : "nome_regione",
                                                            "location" : "coordinate", "lib_du_b_vote" : "etichetta_seggio_elettorale",
                                                            "scrutin_code" : "codice_scrutinio"}, inplace=True)
new_ordered_columns = ["codice_regione","nome_regione","codice_dipartimento","nome_dipartimento","codice_EPCI", "nome_EPCI","codice_comune","nome_comune",
                       "codice_circoscrizione","nome_circoscrizione","codice_seggio_elettorale","iscritti","astenuti","% astenuti/iscritti","votanti",
                       "% votanti/iscritti","bianche","% bianche/iscritti","% bianche/votanti","nulle","% nulle/iscritti","% nulle/votanti","espressi",
                       "% espressi/iscritti","% espressi/votanti","pannello_numero","genere","cognome","nome","partito","voti","% voti/iscritti","% voti/espressi",
                       "coordinate","etichetta_seggio_elettorale","codice_scrutinio"]
df_risultati_elezioni_presidenziali_1_turno = df_risultati_elezioni_presidenziali_1_turno.reindex(columns=new_ordered_columns)
df_risultati_elezioni_presidenziali_1_turno.to_csv(file_path_work_risultati_delle_elezioni_presidenziali_1_turno, index=False)
#df_risultati_elezioni_presidenziali_1_turno.head(10)

In [ ]:
# Caricamento e cleaning dataset risultati_elezioni_presidenziali_2_turno

df_risultati_elezioni_presidenziali_2_turno = filtra_dataset_per_regioni(file_path_risultati_delle_elezioni_presidenziali_2_turno,
                                                                         dtype={"Code du département": str, "Code de la circonscription": str,
                                                                                "Code de la commune": str, "Code du b.vote": str, "Code Officiel EPCI": str,
                                                                                "Exprimés": int})
df_risultati_elezioni_presidenziali_2_turno['% Voix/Exp'] = df_risultati_elezioni_presidenziali_2_turno['% Voix/Exp'].apply(float_converter)
df_risultati_elezioni_presidenziali_2_turno['% Voix/Exp'] = df_risultati_elezioni_presidenziali_2_turno['% Voix/Exp'].astype(float)
df_risultati_elezioni_presidenziali_2_turno.rename(columns={"Code du département" : "codice_dipartimento", "Libellé du département" : "nome_dipartimento",
                                                            "Code de la circonscription" : "codice_circoscrizione",
                                                            "Libellé de la circonscription" : "nome_circoscrizione", "Code de la commune" : "codice_comune",
                                                            "Libellé de la commune" : "nome_comune", "Code du b.vote" : "codice_seggio_elettorale",
                                                            "Inscrits" : "iscritti", "Abstentions" : "astenuti", "% Abs/Ins" : "% astenuti/iscritti",
                                                            "Votants" : "votanti", "% Vot/Ins" : "% votanti/iscritti", "Blancs" : "bianche",
                                                            "% Blancs/Ins" : "% bianche/iscritti", "% Blancs/Vot" : "% bianche/votanti",
                                                            "Nuls" : "nulle", "% Nuls/Ins" : "% nulle/iscritti", "% Nuls/Vot" : "% nulle/votanti",
                                                            "Exprimés" : "espressi", "% Exp/Ins" : "% espressi/iscritti", "% Exp/Vot" : "% espressi/votanti",
                                                            "N°Panneau" : "pannello_numero", "Sexe" : "genere", "Nom" : "cognome", "Prénom" : "nome",
                                                            "Nuance" : "partito", "Voix" : "voti", "% Voix/Ins" : "% voti/iscritti",
                                                            "% Voix/Exp" : "% voti/espressi", "Code Officiel EPCI" : "codice_EPCI", "Nom Officiel EPCI" : "nome_EPCI",
                                                            "Code Officiel Région" : "codice_regione", "Nom Officiel Région" : "nome_regione",
                                                            "location" : "coordinate", "lib_du_b_vote" : "etichetta_seggio_elettorale",
                                                            "scrutin_code" : "codice_scrutinio"}, inplace=True)
new_ordered_columns = ["codice_regione","nome_regione","codice_dipartimento","nome_dipartimento","codice_EPCI", "nome_EPCI","codice_comune","nome_comune",
                       "codice_circoscrizione","nome_circoscrizione","codice_seggio_elettorale","iscritti","astenuti","% astenuti/iscritti","votanti",
                       "% votanti/iscritti","bianche","% bianche/iscritti","% bianche/votanti","nulle","% nulle/iscritti","% nulle/votanti","espressi",
                       "% espressi/iscritti","% espressi/votanti","pannello_numero","genere","cognome","nome","partito","voti","% voti/iscritti","% voti/espressi",
                       "coordinate","etichetta_seggio_elettorale","codice_scrutinio"]
df_risultati_elezioni_presidenziali_2_turno = df_risultati_elezioni_presidenziali_2_turno.reindex(columns=new_ordered_columns)
df_risultati_elezioni_presidenziali_2_turno.to_csv(file_path_work_risultati_delle_elezioni_presidenziali_2_turno, index=False)
#df_risultati_elezioni_presidenziali_2_turno.head(10)

In [ ]:
# Caricamento e cleaning dataset disoccupazione

df_disoccupazione = filtra_dataset_per_regioni(file_path_dati_sulla_disoccupazione)
df_disoccupazione.rename(columns={"Période (Trimestre)": "periodo_trimestre","Date":"data","Catégorie":"categoria",
                                  "Tranche d'âge":"fascia_eta","Nb moyen demandeur emploi":"numero_medio_persone_cercanti_lavoro",
                                  "Nb moyen demandeur emploi Homme":"numero_medio_uomini_cercanti_lavoro",
                                  "Nb moyen demandeur emploi Femme":"numero_medio_donne_cercanti_lavoro",
                                  "Différence Nb demandeur emploi trimestre précédent":"differenza_con_cercanti_lavoro_trimestre_prec",
                                  "Nb moyen entrées":"numero_medio_entrate","Nb moyen sorties":"numero_medio_uscite",
                                  "Nb moyen demandeur emploi longue durée":"numero_medio_persone_cercanti_lavoro_lungo_termine",
                                  "Code Officiel Département":"codice_dipartimento","Code Officiel Région":"codice_regione",
                                  "Nom Officiel Région":"nome_regione","Code Iso 3166-3 Zone":"Code Iso 3166-3 Zona",
                                  "Nom Officiel Département":"nome_dipartimento"}, inplace=True)
df_disoccupazione.drop(["periodo_trimestre", "categoria", "fascia_eta", "numero_medio_uomini_cercanti_lavoro", "numero_medio_donne_cercanti_lavoro",
                        "numero_medio_entrate", "numero_medio_uscite", "numero_medio_persone_cercanti_lavoro_lungo_termine", "Code Iso 3166-3 Zona"],
                        axis=1, inplace=True)
new_ordered_columns = ["data","nome_regione","codice_regione","nome_dipartimento","codice_dipartimento","numero_medio_persone_cercanti_lavoro",
                       "differenza_con_cercanti_lavoro_trimestre_prec"]
df_disoccupazione = df_disoccupazione.reindex(columns=new_ordered_columns)
df_disoccupazione.to_csv(file_path_work_dati_sulla_disoccupazione, index=False)
#df_disoccupazione.head(10)

In [ ]:
# Caricamento e cleaning dataset andamento_prezzi_carburanti(elaborazione lenta puo impiegare anche 1 min)

df_andamento_prezzi_carburanti = filtra_dataset_per_regioni(file_path_andamento_dei_prezzi_dei_carburanti,
                                                            dtype={"Code officiel commune ou arrondissement": str, "Code Officiel Département": str})
df_andamento_prezzi_carburanti.rename(columns={"Timetable":"orario","Identifiant":"identificativo","Code Postal":"cap","Présence":"presenza","Adresse":"indirizzo",
                                               "Nom Officiel Commune / Arrondissement Municipal Majuscule":"nome_comune",
                                               "Code officiel commune ou arrondissement":"codice_comune","Nom Officiel Département":"nome_dipartimento",
                                               "Code Officiel Département":"codice_dipartimento","Nom Officiel Région":"nome_regione",
                                               "Code Officiel Région":"codice_regione","Nom Officiel EPCI":"nome_EPCI","Code Officiel EPCI":"codice_EPCI",
                                               "Automate 24-24":"automatico","Carburant":"carburante","Rupture":"interruzione","Mise à jour":"data",
                                               "Prix Gazole":"prezzo_diesel","Prix SP95":"prezzo_SP95","Prix SP98":"prezzo_SP98","Prix GPLc":"prezzo_GPLc",
                                               "Prix E10":"prezzo_E10","Prix E85":"prezzo_E85","Services":"servizi","Marque":"marca","Nom":"nome",
                                               "Geo Point":"punto_geografico"}, inplace=True)
new_ordered_columns = ["orario","identificativo","cap","presenza","indirizzo","nome_regione","codice_regione","nome_dipartimento","codice_dipartimento","nome_EPCI",
                       "codice_EPCI","nome_comune","codice_comune","automatico","carburante","interruzione","data","prezzo_diesel","prezzo_SP95",
                       "prezzo_SP98","prezzo_GPLc","prezzo_E10","prezzo_E85","servizi","marca","nome","punto_geografico"]
df_andamento_prezzi_carburanti = df_andamento_prezzi_carburanti.reindex(columns=new_ordered_columns)
df_andamento_prezzi_carburanti['data'] = df_andamento_prezzi_carburanti['data'].map(substring_datetime)
df_andamento_prezzi_carburanti['data'] = pd.to_datetime(df_andamento_prezzi_carburanti['data'])
df_andamento_prezzi_carburanti = df_andamento_prezzi_carburanti[((df_andamento_prezzi_carburanti['data'].dt.year == 2022) &
                                                          (df_andamento_prezzi_carburanti['data'].dt.month >= 4)) |
                                                          (df_andamento_prezzi_carburanti['data'].dt.year == 2023)]
df_andamento_prezzi_carburanti.to_csv(file_path_work_andamento_dei_prezzi_dei_carburanti, index=False)
#df_andamento_prezzi_carburanti.head(10)

In [ ]:
# Caricamento e cleaning dataset reddito

df_reddito_dipartimenti = filtra_dataset_per_regioni(file_path_reddito_per_dipartimenti)
df_reddito_dipartimenti.rename(columns={"Nom Officiel Région":"nome_regione", "Libellé du département" : "nome_dipartimento", "Revenu": "reddito_pro_capite"},
                               inplace=True)
df_reddito_dipartimenti['reddito_pro_capite'] = df_reddito_dipartimenti['reddito_pro_capite'].map(thousands_converter)
df_reddito_dipartimenti.to_csv(file_path_work_reddito_per_dipartimenti, index=False)
#df_reddito_dipartimenti.head(10)

In [ ]:
# Caricamento dataset inflazione

df_inflazione = pd.read_csv(file_path_inflazione, sep=';')
df_inflazione.to_csv(file_path_work_inflazione, index=False)
#df_inflazione.head(10)